In [1]:
import numpy as np
import math

Рассчет вероятностей успеха, коллизии, пустого RAW-слота. Параметр W_0

In [74]:
W_0 = 14
k = 6
n = 7

In [75]:
p_s = []
for l in range (0, k + 1):
    p_s.append(n*(W_0-l-1)**(n-1)/W_0**n)
p_c = []
t = 0
for l in range (0, k + 1):   #в l + 1 слоте i станций && в последующих все остальные
    for i in range (1, n + 1):  
        t +=  math.factorial(n)/math.factorial(n-i)/math.factorial(i)*(W_0-l-1)**(n-i)/W_0**n
    p_c.append(t)
    t = 0
p_e = []
p_e.append((W_0-k-1)**n/W_0**n)

Расчет вероятностей успеха, коллизии, пустого RAW-слота. Метод доступа p-CSMA

In [7]:
p = 0.5
k = 5
n = 5

In [77]:
p_suc = []
for l in range (0, k + 1):
    p_suc.append((1-p)**(l*n)*n*(p)*(1-p)**(n-1))
p_col = []
t = 0

for l in range (0, k + 1):
    for i in range (2, n + 1):
        t += ((1 - p)**(n*l)*math.factorial(n)/math.factorial(i)/math.factorial(n-i)*p**i*(1-p)**(n-i))
    p_col.append(t)
    t = 0
    
p_empty = []
p_empty.append((1-p)**((k+1)*n))

In [78]:
sum(p_col + p_empty + p_suc)

0.9999999999999997

Цепи Маркова

In [79]:
W_0 = 16
k = 6 #длительность слота
n = 0
N = 2 # максимальное число станций

In [80]:
Tper = (1024 + k*52)/0.1
#Положим, как и в работах, Traw/Tper = 0.1 = (Ts + k * Tempty)/Tper = 0.1
#Tempty = 52us
#Ts = Tc = 1024 us

In [15]:
def P_s(k, W_0, n):
    p_s = 0
    if n == 0:
        return 0
    for l in range (0, k + 1):
        p_s += (n*(W_0-l-1)**(n-1)/W_0**n)
    return p_s

In [58]:
summary = 0
for j in range (1, N + 1):
        summary += P_s(k, W_0, j)
        
 #sum  = lambda * Tper * N
_lambda = summary/(N + 1)/N/Tper
_lambda = 4.69169115218968e-05

In [81]:
q = 1-np.exp(Tper*(-_lambda)) # приход кадра

In [82]:
def Cbin(m, x, N):
    if (m + x > N):
        return 0
    return math.factorial(N - x)/math.factorial(N - x - m)/math.factorial(m)*q**m*(1-q)**(N - x -m)

In [83]:
def P_c(k, W_0, n):
    p_c = 0
    t = 0
    for l in range (0, k + 1):
        for i in range (2, n + 1):  
            t +=  math.factorial(n)/math.factorial(n-i)/math.factorial(i)*(W_0-l-1)**(n-i)/W_0**n
        p_c += (t)
        t = 0
    return p_c
def P_e(k, W_0, n):
    return (W_0-k-1)**n/W_0**n

In [84]:
def next_step(N, W_0, k): #модель, когда сначала race, затем станции
    P = np.zeros((N + 1, N + 1))
    for i in range (0, N + 1):
        for j in range (0, N + 1):
            if (i - j == 1):
                P[i][j] = P_s(k, W_0, i)*Cbin(0, i - 1, N)
            elif (i - j > 1):
                P[i][j] = 0
            else:
                P[i][j] = (P_e(k, W_0, i) + P_c(k, W_0, i))*Cbin(j-i, i, N) + P_s(k, W_0, i)*Cbin(j  + 1 - i, i - 1, N)
    return P

In [86]:
def next_step_come(N, W_0, k):    #модель, когда сначала приходят станции, затем race
    P = np.zeros((N + 1, N + 1))
    for i in range (0, N + 1):
        for j in range (0, N + 1):
            if (i - j == 1):
                P[i][j] = P_s(k, W_0, i)*Cbin(0, i, N)
            elif (i - j > 1):
                P[i][j] = 0
            else:
                P[i][j] = (P_e(k, W_0, j) + P_c(k, W_0, j))*Cbin(j-i, i, N) + P_s(k, W_0, j + 1)*Cbin(j  + 1 - i, i, N)
    return P



# Найдем стационарное состояние цепи Маркова



Первым способом решим явно уравнение

$$ \psi * P = \psi$$

In [88]:
N = 10
eq = np.zeros(N + 1)
eq[N] = 1
ones = np.ones(N + 1)

P = next_step(N,W_0, k).T
for i in range (0, N + 1):
    for j in range (0, N + 1):
        if (i == j):
            P[i][j] += -1
P[N] = ones
x =  np.linalg.solve(P, eq)
np.set_printoptions(precision=2)
x

array([2.56e-16, 2.01e-13, 8.54e-11, 1.66e-08, 1.61e-06, 7.97e-05,
       2.03e-03, 2.62e-02, 1.62e-01, 4.32e-01, 3.78e-01])

Второй способ

In [90]:
N = 5
k = 6
W_0 = 16
A = P = next_step(N,W_0, k)
for i in range (0, 10*N):
    A = A.dot(P)
A #P*P....*P

array([[3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01],
       [3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01],
       [3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01],
       [3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01],
       [3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01],
       [3.48e-05, 1.75e-03, 2.84e-02, 1.82e-01, 4.51e-01, 3.37e-01]])

In [32]:
#Очевидно, что при умножении нашего вектора (1,0,0) для системы N = 2, будет распределение-строка матрицы выше

In [91]:
N = 5
k = 6
W_0 = 16
A = P = next_step_come(N,W_0, k)
for i in range (0, 10*N):
    A = A.dot(P)
A #P*P....*P

array([[0.  , 0.02, 0.15, 0.4 , 0.36, 0.06],
       [0.  , 0.02, 0.15, 0.4 , 0.36, 0.06],
       [0.  , 0.02, 0.15, 0.4 , 0.36, 0.06],
       [0.  , 0.02, 0.15, 0.4 , 0.36, 0.06],
       [0.  , 0.02, 0.15, 0.4 , 0.36, 0.06],
       [0.  , 0.02, 0.15, 0.4 , 0.36, 0.06]])

# Валидация экспериментов

In [34]:
import numpy as np
import pandas as pd
import itertools
from collections import Counter

In [35]:
k = 5
active = 6
W_0 = 16

Ручной симулятор для числа станций = 6

In [36]:
def theoretical_success(active, k, W_0):
    theor_suc = pd.DataFrame(columns = ['Min_Backoff','Occurance' , 'End'])
    backoff = np.arange(0, W_0)
    ctrs = []
    minval = []
    minoc = []
    
    for i in range(W_0):
        for j in range(W_0):
            for b in range(W_0):
                for l in range(W_0):
                    for y in range(W_0):
                        for p in range(W_0):
                            cur = []
                            cur.append(i)
                            cur.append(j)
                            cur.append(b)
                            cur.append(l)
                            cur.append(y)
                            cur.append(p)
                            minimum = min(cur)
                            ctrs.append(minimum)
                            minimum_occurance = cur.count(minimum)
                            minval.append(minimum_occurance)
                            if (minimum <= k):
                                if (minimum_occurance == 1):
                                    minoc.append('s')                
                                else:
                                    minoc.append('c') 
                            else: 
                                minoc.append('e') 
    theor_suc['Min_Backoff'] = ctrs
    theor_suc['Occurance'] = minval
    theor_suc['End'] = minoc
    return theor_suc

In [37]:
th_s = theoretical_success(6, 1, W_0)

In [38]:
 th_s[th_s['End'] == 's'].count()

Min_Backoff    7783194
Occurance      7783194
End            7783194
dtype: int64

In [39]:
N = 49
k = 6
W_0 = 16
n = np.arange(1, N + 1)
success = []
for i in range (1, N + 1):
    p = (P_s(k,W_0, i))
    success.append(p)
success.insert(0, 0)
success       #вероятность успеха от числа активных станций за 1 RAW-слот

[0,
 0.4375,
 0.65625,
 0.7587890625,
 0.7998046875,
 0.8084297180175781,
 0.8001651763916016,
 0.7832485884428024,
 0.7620516121387482,
 0.7388906567939557,
 0.7149906565609854,
 0.6909974796051301,
 0.6672500429083357,
 0.6439246510998986,
 0.6211114711690482,
 0.5988550063032839,
 0.5771754947495557,
 0.5560802140823761,
 0.5355694498931225,
 0.5156396478261369,
 0.4962850806008059,
 0.47749873313280455,
 0.4592727765387543,
 0.44159882630306946,
 0.424468087311247,
 0.40787143968178396,
 0.39179949366466316,
 0.3762426283860631,
 0.3611910221388627,
 0.3466346782071863,
 0.3325634482708677,
 0.31896705442125906,
 0.3058351102915978,
 0.2931571415315424,
 0.2809226057150662,
 0.26912091170016,
 0.2577414384244502,
 0.24677355310554716,
 0.23620662880958365,
 0.22603006135123505,
 0.21623328549102855,
 0.20680579039962527,
 0.19773713436329285,
 0.189016958709584,
 0.18063500093709522,
 0.17258110703795743,
 0.16484524300634307,
 0.15741750553069894,
 0.15028813187160547,
 0.14344750

In [40]:
data = pd.read_csv("D:\IITP\Results\\active_log.txt", sep = ',', header = None)
data.columns = ["N_active_before", "N_active_after", "Status"]
data[data["N_active_before"] == 30]

,N_active_before,N_active_after,Status
66,30,30,c
67,30,29,s
178,30,29,s
257,30,29,s
376,30,30,c
...,...,...,...
9667,30,29,s
9703,30,30,c
9867,30,29,s
9944,30,30,c


In [41]:
data.groupby('N_active_before').count()

,N_active_after,Status
N_active_before,,
0,55,55
1,28,28
2,32,32
3,33,33
4,38,38
5,47,47
6,56,56
7,70,70
8,94,94


In [42]:
data['Total_freq'] = data.groupby('N_active_before').count()
data

ValueError: Wrong number of items passed 2, placement implies 1

In [ ]:
res = pd.DataFrame()
res['Frequency'] = data.groupby('N_active_before').count()['N_active_after']
res['Success'] = data[data['Status'] == ' s'].groupby('N_active_before').count()['N_active_after']
res

In [ ]:
res['Success prob'] = res['Success']/res['Frequency']
res['Success prob_theor'] = success
res

In [ ]:
res

In [ ]:
data.groupby(['N_active_before', 'Status']).count()

In [ ]:
writer = pd.ExcelWriter("D:\IITP\Results\Success_prob.xlsx", engine='xlsxwriter')

In [ ]:
res.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format': '0%'})

# Note: It isn't possible to format any cells that already have a format such
# as the index or headers or any cells that contain dates or datetimes.

# Set the column width and format.
worksheet.set_column('B:B', 18, format1)

# Set the format but not the column width.
worksheet.set_column('C:C', None, format2)

# Close the Pandas Excel writer and output the Excel file.
writer.save()